<a href="https://colab.research.google.com/github/JustAaron/CS301/blob/main/Aaron_Lee_CS301_101_Group10_Project_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Shows data about the GPU being used
# If a GPU is not being used, please change the runtime type to 'GPU'
# As with other types of Machine Learning, a GPU greatly increases the speed of training and testing
!nvidia-smi

Sun Nov 28 02:08:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Installs the necessary libraries

# Installs the latest version of Haystack (with dependencies
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

# Needed to download documents
!pip install wikipedia

# Note: After running this code, the Google Colab should be manually restarted so that the libraries are updated.

     |████████████████████████████████| 2.5 MB 5.3 MB/s 
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-5qt0a3_8
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-5qt0a3_8
     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 14.1 MB 6.2 MB/s 
     |████████████████████████████████| 2.5 MB 30.7 MB/s 
     |████████████████████████████████| 51 kB 520 kB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 79 kB 7.2 MB/s 
     |████████████████████████████████| 321 kB 50.6 MB/s 
     |████████████████████████████████| 365 kB 50.2 MB/s 
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 981 kB 36.1 MB/s 
     |████████████████████████████████| 3.0 MB 35.1 MB/s 
     |████████████████████████████████| 78 kB 6.1 MB/s 
     |████████████████████████████████| 5.6 MB 19.3 MB/s 
     |█████████

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=8c44b5b2a177ac66141b9bf71dcac5de96879a45073a68080f84b4083db7d356
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [1]:
# Imports only necessary files from the libraries

# Utility to print the answer from the Reader node
from haystack.utils import print_answers

# Imports the Retriever node (the node that will filter the documents to ideally only relevant ones)
# Because the Reader node is resource intensive, it is helpful to narrow down the documents beforehand with a more basic method
from haystack.nodes import ElasticsearchRetriever

# Imports two types of Reader nodes (the node that will interpret some given documents and be able to output an answer)
from haystack.nodes import FARMReader, TransformersReader

# Module to get articles from wikipedia
import wikipedia

/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
# In Colab, ElasticSearch must be downloaded from source
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.9.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ElasticSearch has started
! sleep 30

In [3]:
# Connect to Elasticsearch
from haystack.document_stores import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

In [4]:
# Downloads and stores the Wikipedia articles to be stored in ElasticSearch
wiki_titles = ["Taj_Majal", "Statue_of_Liberty", "Burj_Khalifa", "Great_Wall_of_China", "Eiffel_Tower", "Berlin_Wall", "Machu_Picchu", "Stonehenge", "Mount_Rushmore_National_Memorial", "Colosseum", "Great_Pyramid_of_Giza", "One_World_Trade_Center", "Empire_State_Building", "White_House", "Petra", "Large_Hadron_Collider", "Hagia_Sophia", "Golden_Gate_Bridge", "Panama_Canal", "Angkor_Wat", "Buckingham_Palace", "Big_Ben", "Guantanamo_Bay_detention_camp", "Christ_the_Redeemer_(statue)", "Pantheon,_Rome", "Parthenon", "Disneyland", "Colossus_of_Rhodes", "New_York_Stock_Exchange", "Wembley_Stadium"]
dicts = []

document_store.delete_documents()  # clears any pre-existing documents that may be lingering in the database from the last iteration

for w_title in wiki_titles:
  wiki_page = wikipedia.page(w_title, auto_suggest=False)
  dicts.append({'content' : wiki_page.content, 'meta' : {'name' : w_title}})
document_store.write_documents(dicts)

In [5]:
# Create Retriever node using ElasticSearch DocumentStore
retriever = ElasticsearchRetriever(document_store=document_store)

In [6]:
# Creates 2 pipelines (one for each type of Reader)
reader_farm = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
reader_transformer = TransformersReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

INFO - haystack.modeling.logger -  ML Logging is turned off. No parameters, metrics or artifacts will be logged to MLFlow.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 2 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0    0 
INFO - haystack.modeling.infer -  /w\  /w\
INFO - haystack.modeling.infer -  /'\  / \
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1


In [7]:
# Creates the ExtractiveQA Pipelines (FARMReader and TransformerReader)
from haystack.pipelines import ExtractiveQAPipeline
pipe_extractqa_farm = ExtractiveQAPipeline(reader_farm, retriever)
pipe_extractqa_transformer = ExtractiveQAPipeline(reader_transformer, retriever)

In [8]:
# Query and Parameters (for ExtractiveQA)
# For Retrievers, top_k is how many documents to return
# For Readers, top_k is how many answers to return
query = "Where is Mount Rushmore?"
params_extractqa = {"Retriever": {"top_k": 3}, "Reader": {"top_k": 5}}

In [9]:
# Runs the query with parameters on ExtractiveQA pipeline (FARM)
prediction_1 = pipe_extractqa_farm.run(
    query=query, params=params_extractqa
)
print_answers(prediction_1, details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]/usr/local/lib/python3.7/dist-packages/haystack/modeling/model/prediction_head.py:455: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|██████████| 2/2 [00:01<00:00,  1.11 Batches/s]


Query: Where is Mount Rushmore?
Answers:
[   {   'answer': 'South Dakota',
        'context': 'h Reagan and Dwight D. Eisenhower at 11%.\n'
                   '\n'
                   '\n'
                   '== Tourism ==\n'
                   "Tourism is South Dakota's second-largest industry, and "
                   "Mount Rushmore is the state's top tou",
        'score': 0.7983101606369019},
    {   'answer': 'Black Hills',
        'context': 'at the Sioux had not received just compensation for their '
                   'land in the Black Hills, which includes Mount Rushmore. '
                   'The court proposed $102 million as c',
        'score': 0.6652486026287079},
    {   'answer': 'Black Hills near Keystone, South Dakota',
        'context': ' (Lakota: Tȟuŋkášila Šákpe, or Six Grandfathers) in the '
                   'Black Hills near Keystone, South Dakota. Sculptor Gutzon '
                   "Borglum created the sculpture's desig",
        'score': 0.61777114868164

In [10]:
# Runs the query with parameters on ExtractiveQA pipeline (Transformers)
prediction_2 = pipe_extractqa_transformer.run(
    query=query, params=params_extractqa
)
print_answers(prediction_2, details="medium")

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/question_answering.py:316: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  fw_args = {k: torch.tensor(v, device=self.device) for (k, v) in fw_args.items()}



Query: Where is Mount Rushmore?
Answers:
[   {   'answer': 'South Dakota',
        'context': 'th Reagan and Dwight D. Eisenhower at 11%.\n'
                   '\n'
                   '\n'
                   '== Tourism ==\n'
                   "Tourism is South Dakota's second-largest industry, and "
                   "Mount Rushmore is the state's top tour",
        'score': 0.5474766492843628},
    {   'answer': 'Port Chester, New York',
        'context': 'er of the mountain was Luigi Del Bianco, an artisan and '
                   'stonemason in Port Chester, New York. Del Bianco emigrated '
                   'to the U.S. from Friuli in Italy and was chosen',
        'score': 0.5157399773597717},
    {'answer': '', 'score': 0.4961082738240443},
    {   'answer': 'South Dakota',
        'context': 'n conceived the idea for Mount Rushmore in 1923 to promote '
                   'tourism in South Dakota. In 1924, Robinson persuaded '
                   'sculptor Gutzon Borglum to 